In [1]:
import pandas as pd
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
from datetime import timedelta

In [2]:
df_bitcoin = pd.read_csv("../dataset/BCHAIN-MKPRU.csv")
df_gold = pd.read_csv("../dataset/LBMA-GOLD.csv")

In [3]:
"""
Model 1: 
    No transactions during five years
    Test different combinations of investment ratio
"""
cash = 1000
share = [0, 0]
best = [0, 0, 0]
for i in range(101):
    share[0] = cash*0.98 * (i/100) / df_bitcoin.iloc[0]['Value']
    share[1] = cash*0.99 * (1-(i/100)) / df_gold.iloc[0]['USD (PM)']
    final_asset = df_bitcoin.iloc[-1]['Value'] * share[0] + df_gold.iloc[-1]['USD (PM)'] * share[1]
    if (final_asset > best[0]):
        best[0] = final_asset
        best[1] = i/100
        best[2] = 1-i/100

In [4]:
best

[73097.91072146707, 1.0, 0.0]

In [5]:
money = 1000 * 0.99
for index in df_gold.index:
    row = df_gold.loc[index]
    percent = row['Return']
    money = money * (1+percent)

In [6]:
money * 0.99

1327.8630982938464

In [7]:
"""
A wrapper function that calculates the final amount of assets given different thresholds as inputs
(For Model 2)
"""
def method2(df, maxthres, minthres):
    money = 1000 * 0.98
    cur = 0
    status = 1
    for index in df.index:
        row = df.loc[index]
        percent = row['Return']
        price = row['Value']
        cur += percent
        if status == 1:
            if cur > maxthres:
                status = 0
                money = money * (1+percent) * 0.98
                cur = 0
            else:
                money = money * (1+percent)
        if status == 0:
            if cur < minthres:
                status = 1
                cur = 0     
    return money

In [8]:
"""
Model 2:
    Define two constant thresholds:
    - one positive threshold for determining when to sell
    - one negative threshold for determining when to buy
"""
max_money = 0
for i in tqdm(range(0, 100)):
    for j in range(-20, 0):
        cur = method2(df_bitcoin, i/100,j/100)
        if cur > max_money:
            max_money = cur
            max = i
            min = j

100%|██████████| 100/100 [05:02<00:00,  3.02s/it]


In [9]:
money, max, min

(1341.275856862471, 76, -4)

In [10]:
"""
Model 3:
    Moving Average, Golden Cross and Dead Cross
"""

money = 1000
status = 0
for index in df_bitcoin.index:
    if index != 1825:
        row = df_bitcoin.loc[index]
        rowcompare = df_bitcoin.loc[index+1]
        value = rowcompare['Value']
        x1 = row['MA5']
        x2 = rowcompare['MA5']
        y1 = row['MA10']
        y2 = rowcompare['MA10']
        if x1 > y1 and x2 < y2 and status == 1:    # Dead Cross, signal for sale
            status = 0 
            money = share * value * 0.98
        elif x1 < y1 and x2 > y2 and status == 0:  # Golden Cross, signal for buy
            status = 1
            share = money / value * 0.98

In [11]:
money

587.7797327819005

In [12]:
"""
Model 4:
    Buy Signal appears only when MA5, MA10, MA30 increase simul
    Sell all assets when reached expectation threshold
"""

money = 1000
status = 0
cur = 0
for index in df_bitcoin.index:
    if index != 1825:
        row = df_bitcoin.loc[index]
        rowcompare = df_bitcoin.loc[index+1]
        value = rowcompare['Value']
        x1 = row['Value']
        x2 = rowcompare['Value']
        y1 = row['MA5']
        y2 = rowcompare['MA5']
        a1 = row['MA10']
        a2 = rowcompare['MA10']
        b1 = row['MA30']
        b2 = rowcompare['MA30']
        if status == 0 and y2 > y1 and a2 > a1 and b2 > b1:     # Buy Siganl
            status = 1
            share = money / value * 0.98
            buyprice = value
        if status == 1 and value / buyprice > 1.5:              # Sell Signal
            status = 0 
            money = share * value * 0.98
            cur = 0

In [13]:
money

52733.181739128115

In [14]:
"""
Model 5:
    Different Buying Stages
"""
cash = 1000
money_spend = 0
asset = 0  
percent = 10
for index in df_bitcoin.index:
    if index != 1825:   
        cur = df_bitcoin.loc[index]
        nex = df_bitcoin.loc[index+1]
        cur_value = cur['Value']
        cur_ma5 = cur['MA5']
        cur_ma10 = cur['MA10']
        cur_ma30 = cur['MA30']
        nex_value = nex['Value']
        nex_ma5 = nex['MA5']
        nex_ma10 = nex['MA10']
        nex_ma30 = nex['MA30']
        
        if cur_ma5 > cur_ma10 and nex_ma5 < nex_ma10:
            percent = 20
        if cur_ma10 > cur_ma30 and nex_ma10 < nex_ma30:
            percent = 50
            
        if money_spend < cash * 0.8 and cur_ma5 > cur_ma10 and nex_ma5 > nex_ma10:       # signal for buy
            asset += (cash * 0.20) * 0.98 / nex_value
            money_spend += cash * 0.20
        if money_spend < cash * 0.5 and cur_ma5 > cur_ma10 and cur_ma10 > cur_ma30 and nex_ma10 > nex_ma30:     # signal for bigger buy
            asset += (cash * 0.50) * 0.98 / nex_value
            money_spend += cash * 0.50
        if (money_spend != 0) and (nex_value * asset / money_spend) > 1.50:
            cash = (cash - money_spend) + asset * nex_value * 0.98
            money_spend = 0
            asset = 0
total = cash - money_spend + asset * nex_value

In [15]:
total

38816.763280649044

In [16]:
df_bitcoin = pd.read_csv("BCHAIN-MKPRU_model6.csv")
df_gold = pd.read_csv("LBMA-GOLD_model6.csv")

In [17]:
df_gold.columns = ['Date', 'Value', 'Return', 'MA5', 'MA10']
df_gold = df_gold[29:]
df_bitcoin = df_bitcoin[29:]

for index in df_gold.index:
    row = df_gold.loc[index]
    df_gold.loc[index, "Date"] = datetime.strptime(row['Date'], "%m/%d/%y")
for index in df_bitcoin.index:
    row = df_bitcoin.loc[index]
    df_bitcoin.loc[index, "Date"] = datetime.strptime(row['Date'], "%m/%d/%y")

In [18]:
for index in df_gold.index:
    if index != 29:
        row = df_gold.loc[index-1]
        rowcompare = df_gold.loc[index]
        x1 = row['Value']
        x2 = rowcompare['Value']
        y1 = row['MA5']
        y2 = rowcompare['MA5']
        
        if x1 < y1 and x2 > y2:
            df_gold.loc[index, "decide"] = 1
        elif x1 > y1 and x2 < y2:
            df_gold.loc[index, "decide"] = -1
        else:
            df_gold.loc[index, "decide"] = 0

In [19]:
df_gold = df_gold[1:]
df = df_bitcoin.merge(df_gold, how='outer',on='Date')
golddate = df_gold['Date'].values

In [20]:
"""
Final Model: Model 6
    Combining all the ideas above.
    The idea of Moving Average (MA), different buying stages, self-adapted selling thresholds, 
    and diversification of total assets are applied.
"""
trans = pd.DataFrame(columns = ['Date', 'money_owned', 'money_spend', "bitcoin_value", "type"])
money = 1000
cash = 1000
status = 0
goldstatus = 0
thres = 1.5
sellprice = 1000
money_spend = 0
bitcoin = 0
share = 0
for index in df.index:
    if index <= 1766:
        row = df.loc[index]
        rowcompare = df.loc[index+1]
        value = rowcompare['Value_x']
        date = rowcompare['Date']
        x1 = row['Value_x']
        x2 = rowcompare['Value_x']
        y1 = row['MA5_x']
        y2 = rowcompare['MA5_x']
        a1 = row['MA10_x']
        a2 = rowcompare['MA10_x']
        b1 = row['MA30']
        b2 = rowcompare['MA30']
        if a1 > b1 and a2 < b2:
            thres = 1.5
        if a1 < b1 and a2 > b2:
            thres = 3
            
        # Buying and Selling Bitcoins
        if goldstatus == 0 or goldstatus == -1:            
            if money_spend < money * 0.8 and x1 > y1 and x2 > y2:
                if goldstatus == -1:
                    while date not in golddate:
                        date -= timedelta(days=1) 
                    money = share * df_gold[df_gold['Date'] == date].iloc[0]['Value'] * 0.99
                    goldstatus = 0
                bitcoin += (money * 0.20) / value * 0.98
                money_spend += money * 0.20
                dict = {"Date":date, "money_owned":money, "money_spend":money_spend, "bitcoin_value":value, "type":"buy small"}
                trans = trans.append(dict, ignore_index=True)
                
            if money_spend < money * 0.5 and y1 > a1 and y2 > a2:
                if goldstatus == -1:
                    while date not in golddate:
                        date -= timedelta(days=1) 
                    money = share * df_gold[df_gold['Date'] == date].iloc[0]['Value'] * 0.99
                    goldstatus = 0
                bitcoin += (money * 0.50) / value * 0.98
                money_spend += money * 0.50
                dict = {"Date":date, "money_owned":money, "money_spend":money_spend, "bitcoin_value":value, "type":"buy large"}
                trans = trans.append(dict, ignore_index=True)
            
            if (money_spend != 0) and (value * bitcoin / money_spend) > thres:
                money = (money-money_spend) + bitcoin * value * 0.98
                money_spend = 0 
                bitcoin = 0
                goldstatus = 1
                dict = {"Date":date, "money_owned":money, "money_spend":money_spend, "bitcoin_value":value, "type":"sell"}
                trans = trans.append(dict, ignore_index=True)
        
        # Buying and Selling Golds
        if (goldstatus == 1) and (date in golddate):
            if rowcompare['decide'] == 1:
                share = money / rowcompare['Value_y'] * 0.99
                goldstatus = -1
        
        if (goldstatus == -1) and (date in golddate):
            if rowcompare['decide'] == -1:
                money = share * rowcompare['Value_y'] * 0.99
                goldstatus = 0

In [21]:
total = money + bitcoin * value
total

60063.71003857569